In [1]:
import sqlite3
import sqlalchemy
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import yfinance as yf
from datetime import date
from components.tickers import *

start_date = date.fromisoformat("2015-01-03")

In [2]:
engine = sqlalchemy.create_engine('sqlite:///database.db')

In [5]:
df = pd.DataFrame()
tickerList = delisted.split()

for ticker in tickerList:
    df[ticker] = web.DataReader( ticker, 'stooq', start=start_date).Close

C:\Users\pauli\AppData\Local\Temp\ipykernel_10604\3427797853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ticker] = web.DataReader( ticker, 'stooq', start=start_date).Close
C:\Users\pauli\AppData\Local\Temp\ipykernel_10604\3427797853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ticker] = web.DataReader( ticker, 'stooq', start=start_date).Close
C:\Users\pauli\AppData\Local\Temp\ipykernel_10604\3427797853.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

AttributeError: 'DataFrame' object has no attribute 'Close'

In [6]:
ticker1 = yf.Tickers(yfinance_tickers)
dataframe1 = ticker1.history(
    start=start_date, interval="1d", rounding=True, auto_adjust=True).Close

[*********************100%***********************]  676 of 676 completed


In [7]:
dataframe1 = dataframe1.tz_localize(None)
df = df.tz_localize(None)
companiesFrame = pd.concat([dataframe1, df], axis=1)
companiesFrame = companiesFrame.reset_index()
companiesFrame['Date'] = pd.DatetimeIndex(companiesFrame['Date'])

In [3]:
indexes = yf.Tickers(sectorsTickers)
indexes_data = indexes.history(
    start=start_date, interval="1d", rounding=True, auto_adjust=True).Close
indexes_data = indexes_data.tz_localize(None)
indexes_data = indexes_data.reset_index()
indexes_data["Date"] = pd.DatetimeIndex(indexes_data['Date'])

[*********************100%***********************]  12 of 12 completed


In [4]:
indexes_data.to_sql("indexes", engine, if_exists='replace', index=False)

2083

In [8]:
companiesFrame.to_sql("stockData", engine, if_exists='replace', index=False)

2083

In [14]:
pd.read_sql("SELECT Date, PARR FROM companiesToShort", engine, parse_dates="Date")

,Date,PARR
0,2023-04-04,27.05
1,2023-04-03,29.74
2,2023-03-31,29.20
3,2023-03-30,29.11
4,2023-03-29,29.21
...,...,...
2072,2015-01-09,17.05
2073,2015-01-08,17.07
2074,2015-01-07,16.30
2075,2015-01-06,16.20


In [16]:
pd.read_sql("SELECT Date, A FROM stockData", engine, parse_dates="Date")

,Date,A
0,2015-01-05,37.12
1,2015-01-06,36.54
2,2015-01-07,37.02
3,2015-01-08,38.13
4,2015-01-09,37.85
...,...,...
2071,2023-03-28,132.41
2072,2023-03-29,133.63
2073,2023-03-30,134.27
2074,2023-03-31,138.11


In [ ]:
pd.read_sql("SELECT Date, COUNT(*) AS JustCounting FROM stockData WHERE A<35", engine, parse_dates="Date")

,Date,JustCounting
0,2015-08-21,58


In [ ]:

# AVG
# COUNT
# DISTINCT
# LOWER
# MAX
# MIN
# UPPER
# WHERE
# LIKE
# ORDER BY
# LIMIT
# GROUP BY
# DESC

# UPDATE table SET column = value WHERE condition
# DELETE FROM table WHERE condition

# Data types: BLOB (Binary large object), INTEGER, NUMERIC (special number based info like dates, times etc...), REAL (foat number), TEXT

# NOT NULL is used to make sure that value was represented (text or number)

# SELECT column FROM table WHERE id IN (SELECT id FROM table WHERE condition = value) . First executes in brackets, then combines information

# SELECT column FROM table WHERE id IN (SELECT short_id FROM shorts WHERE short_id = (SELECT id FROM longs WHERE long = name)) ORDER BY column

# SELECT * FROM shows JOIN genres ON shows.id = genres.show_id WHERE title = 'the office'
# SELECT * FROM people WHERE name LIKE 'Steve C%'  % sign tells to choose anything after C

# CREATE INDEX column_index ON shows (column_name) . Makes searches much faster with seperate column index. B-trees

# variable_name = input("ticker: ")
# db.execute("SELECT * FROM table WHERE ticker = ?", variable_name)  dynamic routing

# race condition - many users do the same requests.

# to make race condition less problamatic there are few solutions: BEGIN TRANSACTION, COMMIT, ROLLBACK.
# db.execute("BEGIN TRANSACTION")
# rows = db.execute("SELECT * FROM data WHERE id = ?", id)
# likes = rows[0]["likes"]
# db.execute("UPDATE posts SET likes = ? WHERE id = ?", likes+1, id)
# db.execute("COMMIT")

#  '-- in SQL means ignore after dashes (comments out left code). Single qoute helps if DB code line is formatted, not used ?.

In [ ]:
pd.read_sql("SELECT Date, A FROM stockData GROUP BY A", engine, parse_dates="Date").head(3)

,Date,A
0,2015-09-28,31.42
1,2015-09-29,31.76
2,2015-08-25,32.25


In [ ]:
pd.read_sql("SELECT * FROM Long_positions",
            engine, parse_dates="Date").head()

,index,LXP
0,Date,2022-1-1
1,Beta,0.79
2,ATR%,0.15
3,Size_in_$,1685.0
4,Shrs_qty,152.0
